In [17]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from scipy.stats import norm
from scipy.stats import t
from scipy.stats import ttest_ind, shapiro, f_oneway, mannwhitneyu
from statsmodels.stats.proportion import proportions_ztest

In [3]:
#Сразу накидаем функции
def confidence_interval_sigma(n, x_mean, sigma, gamma=0.95):
    """Function to evaluate confidence interval with known true standard deviation

    Args:
        n (int): sample size
        x_mean (int): sample mean
        sigma (int): true standard deviation
        gamma (float): reliability level 0-1

    Returns:
        float: lower_bound, upper_bound
    """
    alpha = 1 - gamma  # уровень значимости

    z_crit = round(-norm.ppf(alpha/2), 2)  # z критическое

    eps = z_crit * sigma/(n ** 0.5)  # погрешность
    lower_bound = round(x_mean - eps, 2)  # левая (нижняя) граница
    upper_bound = round(x_mean + eps, 2)  # правая (верхняя) граница
    # создаём кортеж из округлённых границ интервала
    confidence_interval = (lower_bound, upper_bound)
    # выводим результат
    print(
        f'Доверительный интервал: {lower_bound, upper_bound}, z-крит: {z_crit}')
    return lower_bound, upper_bound, z_crit

def confidence_interval(n, x_mean, x_std, gamma=0.95):
    """_summary_

    Args:
        n (int): sample size
        x_mean (int): sample mean
        x_std (int, float): sample standard deviation
        gamma (float): reliability level 0-1
    """
    k = n - 1  # число степеней свободы
    alpha = 1 - gamma  # уровень значимости

    t_crit = -t.ppf(alpha/2, k)  # t-критическое

    eps = t_crit * x_std/(n ** 0.5)  # погрешность
    lower_bound = round(x_mean - eps, 2)  # левая (нижняя) граница
    upper_bound = round(x_mean + eps, 2)  # правая (верхняя) граница
    # создаём кортеж из округлённых границ интервала
    confidence_interval = (lower_bound, upper_bound)
    #print(
    #    f'Доверительный интервал: {lower_bound, upper_bound}, t-крит: {t_crit}')
    return lower_bound, upper_bound, t_crit

def proportions_conf_interval(n, x_p, gamma=0.95):
    """Function to evaluate confidence interval for proportion and
    true standard deviation.

    Args:
        n (int): sample size
        x_p (int): sample proportion (% of success)
        gamma (float): reliability level 0-1

    Returns:
        float: lower_bound, upper_bound, sigma
    """
    alpha = 1 - gamma  # уровень значимости
    z_crit = -norm.ppf(alpha/2)  # z критическое
    eps = z_crit * (x_p * (1 - x_p) / n) ** 0.5  # погрешность
    sigma = round((x_p * (1 - x_p))**0.5, 3) #true standard deviation

    lower_bound = round((x_p - eps)*100, 2)  # левая (нижняя) граница
    upper_bound = round((x_p + eps)*100, 2)  # правая (верхняя) граница

    #print(
    #    f'Доверительный интервал: {lower_bound, upper_bound}, z-крит: {z_crit}, sigma: {sigma}')
    return lower_bound, upper_bound, sigma

def diff_proportions_conf_interval(n, x_p, gamma=0.95):
    """Function to evaluate confidence interval for delta of proportion.

    Args:
        n (list): list of samples sizes (for example: n = [a_data['user_id'].count(), b_data['user_id'].count()])
        x_p (list): list of samples proportions (for example: xp = [a_data['converted'].mean(), b_data['converted'].mean()])
        gamma (float, optional): reliability level 0-1

    Returns:
        float: lower_bound, upper_bound (%)
    """

    alpha = 1 - gamma  # уровень значимости
    diff = x_p[1] - x_p[0]  # выборочная разница конверсий групп B и A
    z_crit = -norm.ppf(alpha/2)  # z критическое
    eps = z_crit * (x_p[0] * (1 - x_p[0])/n[0] + x_p[1] *
                    (1 - x_p[1])/n[1]) ** 0.5  # погрешность

    lower_bound = round((diff - eps)*100, 2)  # левая (нижняя) граница
    upper_bound = round((diff + eps)*100, 2)  # правая (верхняя) граница
    # возвращаем кортеж из округлённых границ интервала
    return lower_bound, upper_bound


In [6]:
data = pd.read_csv('../EDA/data/new-site.csv', sep='\t')

In [9]:
data

,site,dwell-time
0,0,2
1,1,48
2,0,15
3,1,16
4,0,38
...,...,...
295,0,22
296,0,7
297,1,144
298,0,89


---

Задание 34
Чему равно суммарное время пребывания на сайте в группах А и В? Ответ вводите без пробелов.

In [11]:
# Group A
data.query("site == 0")['dwell-time'].sum()

16898

In [12]:
# Group B
data.query("site == 1")['dwell-time'].sum()

10032

---

35 Чему равно среднее время пребывание на сайте в группах? Ответ округлите до целых.

In [15]:
# Group A
round(data.query("site == 0")['dwell-time'].mean())

91

In [16]:
# Group B
round(data.query("site == 1")['dwell-time'].mean())

88

---

In [39]:
group_a = data.query("site == 0")['dwell-time']
group_a = group_a.reset_index()
group_a

,index,dwell-time
0,0,2
1,2,15
2,4,38
3,6,571
4,8,233
...,...,...
181,294,39
182,295,22
183,296,7
184,298,89


In [38]:
group_b = data.query("site == 1")['dwell-time']
group_b = group_b.reset_index()
group_b

,index,dwell-time
0,1,48
1,3,16
2,5,169
3,7,38
4,10,336
...,...,...
109,281,187
110,282,11
111,283,70
112,291,100


In [37]:
# Не верно
stats.wilcoxon(group_a, group_b)

WilcoxonResult(statistic=array([3.0000e+00, 3.0695e+03]), pvalue=array([2.04823522e-20, 6.65305108e-01]))

---

Каковы результаты статистического теста на нормальность при уровне значимости alpha = 0.05

In [42]:
alpha = 0.05

In [44]:
_, p = shapiro(group_a)
p < alpha

True

In [45]:
_, p = shapiro(group_b)
p < alpha

True

В обеих группах данные не распределены нормально